In [2]:
#import the necessary libraries

import numpy as np 
import matplotlib.pyplot as plt
import json
import pandas as pd


# Incubation Time and Basic reproduction number

In [4]:
def get_meanIT(genome_type, log_folder) : 
   
    time_id = []
    unique_ids = []
    IT = []
    df_final = pd.read_csv('../../data/seir/simulated/It/'+log_folder+'/host999.csv')
    df_final = df_final[df_final['status']=='R']
    #df_final = df_final[df_final['E_nb']>0]
   
    for i in range(1,999) : 
        df = pd.read_csv('../../data/seir/simulated/It/'+log_folder+'/host'+str(i)+'.csv')
        df_I = df[df['status']=='I']
        df_E = df[df['status']=='S']
        df_I = df_I[df_I['viralParams']==genome_type]
        ids = set(df_I['id'].values)
       
        if len(ids) > 0 : 
            next_ids = []
            for elt in ids : 
                if elt < 4900.0 : 
                    try : 
                        next_ids.append(df_final[df_final['id']==elt]['tau'].values[0])
                    except KeyError : 
                        print('KeyError')
                        continue
            if len(next_ids)>0 : 
                IT.append(next_ids)
    return IT

def get_meanBIN(genome_type, log_folder) : 
   
    time_id = []
    unique_ids = []
    R0 = []
    df_final = pd.read_csv('../data/seir/simulated/Rt/'+log_folder+'/host999.csv')
    df_final = df_final[df_final['status']=='R']
    #df_final = df_final[df_final['E_nb']>0]
    
    infectious_id = list(df_final[df_final['I_id']>0]['I_id'].values)
    r0_s = {}
    for id_ in df_final['id'].values: 
        r0_s[id_]= float(infectious_id.count(id_))
    #print (len(r0_s.keys()))
    
    for i in range(1,999) : 
        df = pd.read_csv('../data/seir/simulated/Rt/'+log_folder+'/host'+str(i)+'.csv')
        df_I = df[df['status']=='I']
        df_E = df[df['status']=='S']
        df_I = df_I[df_I['viralParams']==genome_type]
        ids = set(df_I['id'].values)
       
        if len(ids) > 0 : 
            next_ids = []
            for elt in ids : 
                if elt < 4900.0 : 
                    try : 
                        next_ids.append(r0_s[elt])
                    except KeyError : 
                        print('KeyError')
                        continue
            if len(next_ids)>0 : 
                R0.append(np.mean(next_ids))
    return R0

In [ ]:
Rt_robust = []
Rt_fragile = []
for i in range(50) : 
    log_folder = "low_beta/R0/"+str(i)
    print(log_folder)
    Rt_robust.append(get_meanBIN('R', log_folder))
    Rt_fragile.append(get_meanBIN('F',log_folder))
    
    
Rt_robust_high = []
Rt_fragile_high = []
for i in range(50) : 
    log_folder = "high_beta/R0/"+str(i)
    print(log_folder)
    Rt_robust_high.append(get_meanBIN('R', log_folder))
    Rt_fragile_high.append(get_meanBIN('F',log_folder))
    
    

In [ ]:
low_Rt_f = []
low_Rt_r = []

for i in range(184) : 
     
    mean_list = []
    for list_ in Rt_robust : 
        mean_list.append(list_[i])
    low_Rt_r.append(np.mean(mean_list))
    
        
for i in range(300) : 
     
    mean_list = []
    for list_ in Rt_fragile : 
        mean_list.append(list_[i])
    low_Rt_f.append(np.mean(mean_list))


np.savetxt("seir/Rt/Rt_robust_low.txt",low_Rt_r)
np.savetxt("seir/Rt/Rt_fragile_low.txt",low_Rt_f) 


high_Rt_r = []

for i in range(40) : 
     
    mean_list = []
    for list_ in Rt_robust_high : 
        mean_list.append(list_[i])
    high_Rt_r.append(np.mean(mean_list))



high_Rt_f = []

for i in range(40) : 
     
    mean_list = []
    for list_ in Rt_fragile_high : 
        mean_list.append(list_[i])
    high_Rt_f.append(np.mean(mean_list))
       
np.savetxt("seir/Rt/Rt_robust_high.txt",high_Rt_r)
np.savetxt("seir/Rt/Rt_fragile_high.txt",high_Rt_f)     

In [ ]:
distIT_robust_low = []
distIT_fragile_low = []
for i in range(50) : 
    
    log_folder = "low_beta/"+str(i)
    distIT_robust_low.append(get_meanIT('R', log_folder))
    distIT_fragile_low.append(get_meanIT('F',log_folder))
    print(log_folder)
    
distIT_robust_high = []
distIT_fragile_high = []

for i in range(20) : 
    log_folder = "high_beta/"+str(i)
    distIT_robust_high.append(get_meanIT('R', log_folder))
    distIT_fragile_high.append(get_meanIT('F',log_folder))
    print(log_folder)
    

#Save the data in a csv file.
with open("../../EvoEpi/data/seir/It/distIT_robust_high.json", 'w') as ft : 
    json.dump(distIT_robust_high, ft)
    
with open("../../EvoEpi/data/seir/It/distIT_fragile_high.json", 'w') as ft : 
    json.dump(distIT_fragile_high, ft)
    
with open("../../EvoEpi/data/seir/It/distIT_robust_low.json", 'w') as ft : 
    json.dump(distIT_robust_low, ft)
    
with open("../../EvoEpi/data/seir/It/distIT_fragile_low.json", 'w') as ft : 
    json.dump(distIT_fragile_low, ft)